# Module 1 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID. Make sure you submit both an .ipynb and .html version of your *completed* notebook. You can generate the HTML version using:

> ipython nbconvert [notebookname].ipynb

or use the File menu.

Add any additional standard library imports you need here:

In [1]:
import copy, bisect

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
#       hills      5
~       swamp      7
x       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
....*..
...***.
.###...
..##...
..#..**
....***
.......
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [2]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '#', 'x', 'x', '#', '#'], 
  ['.', '.', '.', '.', '#', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '#', '#', 'x', 'x', '#', '.'], 
  ['.', '.', '.', '#', '#', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.'], 
  ['.', '#', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '#', 'x', '#', '.', '.'], 
  ['.', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '#', '.', '.', '.'], 
  ['.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '#', '#', '#', '.', '.'], 
  ['.', '.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '.', '~', '~', '.', '.', '#', '#', '#', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '#', '#', '#', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '#', '#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.', '.', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '#', '#', 'x', 'x', '#', '.', '~', '~', '~', '~', '.', '.', '.', '#', 'x', '#', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '#', '*', '*', '#', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '#', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '#', '#', '#', '#', '.', '~', '~', '~', '.', '.', '#', 'x', '#', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '.', '~', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '#', '#', '#', '#', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '#', '#', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '#', '#', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '#', '#', '.', '~', '~', '~', '~', '~'], 
  ['#', 'x', '#', '#', '#', '#', '.', '.', '.', '.', '.', 'x', 'x', 'x', '#', '#', 'x', 'x', '.', 'x', 'x', '#', '#', '~', '~', '~', '~'], 
  ['#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', 'x', 'x', '#', '#', '#', '#', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '.', '.']]

<div style="background: khaki; color: darkgoldenrod; border: 2px solid goldenrod; padding: 5px; margin: 10px;">
<strong>Warning</strong>
</div>

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7).

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics, what do you expect the policy would be?

In [3]:
test_world = [
  ['.', '*', 'x', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]
test_world2 = [
  ['.', '.', '.', '~', '~', '~', '*'],
  ['.', '*', '.', '~', '.', '.', '.'],
  ['.', '*', '.', '~', '.', '~', '.'],
  ['.', '*', '.', '.', '.', '~', '.'],
  ['*', '*', '*', '~', '~', '~', '.'],
  ['*', '*', '*', '~', '~', '~', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

**Answer**:  
In the first test world, one would expect the algorithm to traverse across the route through the plains. Not only is the plains the least costly terrain to traverse, the route also provides the shortest distance between starting point and the goal. It is trivially provable that all else equal, the Manhattan distance is the least amount of cost that one route could possibly take.

In the second world, one would expect the algorithm to still stay with the route on the plains that traverse through the swamp. In this case, since swamplands have a very high traversal cost, despite the route through the plains not having a Manhattan-distance, it is still preferrable.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [4]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [5]:
costs = { '.': 1, '*': 3, '#': 5, '~': 7}

## A\* Search Implementation

As Python is an interpreted language, you're going to need to insert all of your helper functions *before* the actual `a_star_search` function implementation. Put those implementations here, along with their documentation. There should be one Markdown cell for the documentation, followed by one Codecell for the implementation. I've included two to get you started.

-----

**Successor Function**  
The successor functions takes three inputs:  
* _loc_: the location in the world for which the successor locations are to be generated. It is represented as a tuple of two integers, which can be considered as a coordinate
* _world_: the world in which the algorithm operates in
* _explored_: either set or list of coordinates which had been explored already

The function returns the four directional successors to the location specified subject to the following three criteria:
1. The successor locations are within bound of the world (i.e. cannot generate negative coordinates or coordinates which is greater than the vertical or horizontal length of the world
1. The terrain as indicated has to be traversable (i.e. cannot be mountains, or x)
1. The locations have not been explored (i.e. not in the `explored` argument)

Along with the coordinates of the successor locations, the movements used to get to these locations are also returned as a tuple of two lists, locations and movements. Note that the `movement` is returned as cartesian (x,y) offsets instead of list indices like the coordinate locations are

In [6]:
def successors(loc, world, explored):
    newLocs = list()
    moves = list()
    for n in range( len(cardinal_moves) ):
        m = cardinal_moves[n]
        x = (m[1]+loc[0],m[0]+loc[1])
        if (0<=x[0]<len(world) and 0<=x[1]<len(world[x[0]])) and \
                (world[x[0]][x[1]] != 'x') and (x not in explored) :
            newLocs.append(x)
            moves.append(m)
    return( (newLocs, moves) )

**Frontier Class**  
The class implments the Frontier used in A\* search. The frontier is a priority queue as according to the cost. Instead of using Python's built-in PriorityQueue class, the programmer implements his own since the built-in class does not allow efficient checking of queue membership nor removal of a specific member without dequeueing. 

The class contains four lists and one integer representing the number of nodes currently stored. The lists hold the information about the nodes yet to be expanded. The four lists specifically hold the:
1. estimated cost of going from the node to the goal, i.e. f(n)
1. geographic coordinate point
1. the movements that lead from the starting point to the current node
1. path-cost of arriving at the current nodes, i.e. g(n)

In addition to the standard `get`, `put`, `empty` function that one would expect with a priority queue, the **`Frontier`** class extends the `put` function to incorporate the following capability:
* Check if the inserted node already exist on the node
  * if exists on frontier at a lower cost, ignore the insertion
  * if exists on frontier at a higher cost, replace the node with the lower cost

The object ensure that the node with the least amount of f(n) are stored at the front of the node, such that `get` could be performed in constant time. This is accomplished through using the `bisect` module which searches a sorted list using binary search in O(lg n) time, and insert the node right in front of the left most node whose cost ≥ the inserted cost.

In [7]:
class Frontier:
    def __init__(self): # object to contain all costs and locations in world
        (self.costs, self.points, self.moves, self.gs) = ([], [], [], []) # empty lists
        self.N = 0 # number of elements

    def empty(self):
        return self.N == 0

    def put(self, cost, pt, mv, g):
        try: # first find if pt already exist on frontier
            ind = self.points.index(pt)
        except ValueError: # if pt is not on the list
            ind = -1

        if ind >= 0: # if already on list
            if cost < self.costs[ind]: # if lower cost, remove existing, and add later
                self.N -= 1
                del self.costs[ind]
                del self.points[ind]
                del self.moves[ind]
                del self.gs[ind]
            else: # if lower cost already on list, do nothing
                return            

        # add to appropriate location based on cost
        ind = bisect.bisect_left(self.costs, cost) # find leftmost value >= cost
        self.N += 1
        self.costs.insert(ind, cost)
        self.points.insert(ind, pt)
        self.moves.insert(ind, mv)
        self.gs.insert(ind, g)

    # Dequeue the node with the smallest cost, which should be at the front of lists
    def get(self):
        if self.N == 0: # error, cannot get element when queue is empty
            raise IndexError('Frontier is empty')
        out = (self.costs[0], self.points[0], self.moves[0], self.gs[0])
        self.N -= 1
        self.costs = self.costs[1:] # remove first element (lowest cost)
        self.points = self.points[1:]
        self.moves = self.moves[1:]
        self.gs = self.gs[1:]
        return out
    
    def __repr__(self): # String representation of the frontier queue
        s = ''
        for n in range(self.N):
            s += repr(self.costs[n]) + ':' + repr(self.points[n]) + ', '
        s = 'len=' + str(self.N) + ' [' + s[:-2] + ']'
        return s

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

**a_star_search**

The `a_star_search` function uses the A\* Search algorithm to solve a navigational problem for an agent in a grid world. It calculates a path from the start state to the goal state and returns the actions required to get from the start to the goal.

* **world** is the starting state representation for a navigation problem.
* **start** is the starting location, `(x, y)`.
* **goal** is the desired end position, `(x, y)`.
* **costs** is a `Dict` of costs for each type of terrain.
* **moves** is the legal movement model expressed in offsets.
* **heuristic** is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


The function returns the offsets needed to get from start state to the goal as a `List`. For example, for the test world:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

Do not make unwarranted assumptions. For example, do not assume the starting point is always `(0, 0)` or that the goal is always in the lower right hand corner. Do not make any assumptions about the movement model beyond the requirement that they be offsets (it could be offets of 2!).

---

**A\* Search Algorithm**  
The function takes six inputs:
1. _world_: The world for which the algorithm finds the optimal route
1. _start_: The coordinate point of the starting location
1. _goal_: The coordinate point of the ending location
1. _costs_: dictionary of costs associated with traversing various terrains
1. _moves_: List of valid movement sets
1. _heuristic_: The heuristic function used to estimate cost of arriving at goal from a node

The implementation is based on the algorithm on p. 84 or fig. 3.14 in the textbook. It iteratively explores nodes from the starting point, adding the path-cost and the heuristic cost as the priority into the frontier, which then are used to discover the optimal route. 

In [8]:
def a_star_search( world, start, goal, costs, moves, heuristic):
    frontier = Frontier() # priority queue for frontier to pull out lowest eval
    explored = set() # explored location, set for quick lookup

    frontier.put(0, start, [], 0)
    while not frontier.empty(): # repeat until no more nodes on frontier
        (c, x, m, g0) = frontier.get() # get node with smallest cost
        if x == goal:
            return m # return if node is the goal
        explored.add(x)
        (pts, moves) = successors(x, world, explored)
        for n,p in enumerate(pts): # iterates through successor nodes
            if p not in explored: # only unexplored nodes
                g = g0 + costs[world[p[0]][p[1]]]
                f = g + heuristic(p, goal)
                frontier.put(f, p, m+[moves[n]], g)

    return None # failed

**pretty_print_solution**

The `pretty_print_solution` function prints an ASCII representation of the solution generated by the `a_star_search`. For example, for the test world, it would take the `world` and `path` and print:

```
v******
v******
v******
>>>>>>v
******v
******v
******G
```

using `v`, `^`, `>`, `<` to represent actions and `G` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters).


**Pretty Print Solution Function**  
The function takes three inputs:
1. _world_: the world for which the route and terrain are to be printed
1. _path_: list of cartesian offsets that encodes the route taken
1. _start_: The starting point of the route

With the above three parameters, the goal location is trivially determined. The function first makes a deep-copy of the input world so as to not modify the original terrain. The movements are represented as ASCII directional arrows, which is encoded in a dictionary indexed by the cartesian offset tuple.

The algorithm replaces the input world encoded by list of list of characters by iteratively applying the offset to the starting point coordinate and overwriting the terrain symbols with the movement directional arrows. The algorithm prints a G at the end point, which is presumed to be the goal.

In [9]:
def pretty_print_solution(world, path, start):
    x = start
    world = copy.deepcopy(world) # deep-copy world to keep input world the same as it is
    movements = {(0,1):'v', (1,0):'>', (0,-1):'^', (-1,0):'<'} # representations of moves
    for p in path: # loop over all movements
        world[x[0]][x[1]] = movements[p] # Change the graphics
        x = (x[0]+p[1], x[1]+p[0]) # move 
    world[x[0]][x[1]] = 'G' # mark the end point as goal
    
    for r in world: # loop over rows of the world to print
        print('\t' + ''.join(r)) # prints each row

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

**Heuristic Function**  
The Heuristic function used in this case is the Manhattan distance. Mathematically, for two points a and b
$$a=(a_{x},a_{y}), b=(b_{x},b_{y})$$
The Manhattan distance is 
$$D_{Manhattan}=\left | a_{x}-b_{x} \right | + \left | a_{y}-b_{y} \right |$$

This Manhattan distance satisfies both admissability and monotonicity requirement of a heuristic function. The Manhattan distance measures the shortest possible traversal assuming all terrains encountered are plains. In reality, there would be harder terrains, which means the heuristic will never overestimate the actual cost. With Manhattan distance the triangle inequality is satisfied as the distance is equal to the two sides of the triangle. This means this particular heuristic function would work in A\* search.

In [10]:
def heuristic(loc, goal):
    return( abs(goal[0]-loc[0]) + abs(goal[1]-loc[1]) )

In [11]:
test_path = a_star_search( test_world, (0, 0), (6, 6), costs, cardinal_moves, heuristic)
print test_path

[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


In [12]:
pretty_print_solution( test_world, test_path, (0, 0))

	v*x****
	v******
	v******
	>>>>>>v
	******v
	******v
	******G


In [13]:
test_path2 = a_star_search( test_world2, (0, 0), (6, 6), costs, cardinal_moves, heuristic)
print test_path2

[(1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, -1), (0, -1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]


In [14]:
pretty_print_solution( test_world2, test_path2, (0, 0))

	>>v~~~*
	.*v~>>v
	.*v~^~v
	.*>>^~v
	***~~~v
	***~~~v
	******G


In [15]:
full_path = a_star_search( full_world, (0, 0), (26, 26), costs, cardinal_moves, heuristic)
print full_path

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]


In [16]:
pretty_print_solution( full_world, full_path, (0, 0))

	v....**********............
	v......*********..xxxxxxx..
	v...xx***********xxx###xx##
	v...#xxx****~~*****..##xx#.
	v..##xx**..~~~~***...#xxx#.
	v###xx##....~~~~~.....#x#..
	v##xx##....#xxx~~~.....#...
	v.#####......#xxx~~~..###..
	v..###......##xx.~~..###...
	v..~~~..###xxxx...~.###....
	v.~~~~~.##xxx#.....#xxx#...
	v~~~~~..#xx#....~~..#xx#...
	v~~~~..##xx#.~~~~...#x#....
	v~~~~..#**#....~~~~..#.....
	v...x..****####.~~~..#x#...
	v..xxx******xxx##.~.#xx#...
	v.xx**********xxx..xxx.....
	v..xx***********xxxx.......
	v..xxx********...##........
	v...xxx******..........~~~~
	v.####xx*****.x.....~~~~~~~
	v...###xxx**xx......~~~~~~~
	>>>v..###xxxx....##..~~~~~~
	.##>v#####.....##xx##.~~~~~
	#x##v#.....xxx##xx.xx##~~~~
	#xxxv.....##xxxx####xxx~~~~
	##..>>>>>>>>>>>>>>>>>>>>>>G


# Advanced/Future Work

*This section is not required but it is well worth your time to think about the task*

Write a *general* `state_space_search` function that could solve any state space search problem using Depth First Search. One possible implementation would be to write `state_space_search` as a general higher order function that took problem specific functions for `is_goal`, `successors` and `path`. You would need a general way of dealing with states, perhaps as a `Tuple` representing the raw state and metadata: `(<state>, <metadata>)`.